<a href="https://colab.research.google.com/github/robpot891/spicystager/blob/master/PySC2_model_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySc2 machine learning model trainer for Google Colaboratory.

**Author**: Paul Steven Conyngham

**Contributors**: Paul Steven Conyngham, Frank He, William Xu


This Google Colab Notebook is setup to get a basic version of the DQN algorithm training on Colabs. We could have compressed all the necessary steps into a simple bash script file, however we have left them all here so you can see  the steps required to get PySc2 working on Colabs.

Feel free to make a copy of this notebook & modify it for your own PySC2 machine learning purposes.








Steps to get started:

1.   First off click **File**, on the menu above & **Save a copy in drive**, this will copy an instance to your google drive, to allow you to start running it!

2.   Once the notebook has finished copying, go to your fresh copy and click **"connect"** in the top right hand corner of Colabs. This connects your computer to a power Virtual Machine sitting in Google's cloud.




How to run it:


**For those new to Colaboratory**, there are two types of "cell blocks" **text** (like the one you are reading right now) and **code**. For code Cell blocks just click the little play button in order to get the code to run. The play button looks like this:

![alt text](https://image.ibb.co/i4sxHH/Screen_Shot_2018_04_10_at_3_04_50_pm.png)

Or you can simply click the "**Runtime**" menu button up above, and click "**Run all**", sit back & watch colabs go to work.



Also, for those of you looking to dive a little deeper into Colabs, feel free to check out Google's official intro to Colaboratory here:

[Official colab usage documentation](https://colab.research.google.com/notebooks/welcome.ipynb)

One final sidenote: it would be amazing to be able to render here somehow

## Run all the stuff below (takes about 10 minutes) to setup pysc2 on colabs

Go Grab a coffee whilst PySc2 installs.



## [start of installation section]

In [0]:

!pip3 install -U PyDrive
!pip3 install https://download.pytorch.org/whl/cu80/torch-0.3.1-cp36-cp36m-linux_x86_64.whl && pip install torchvision
!apt-get -qq install -y libsm6 libxext6 && pip3 install -U opencv-python
!apt-get install -y xvfb python-opengl
!apt-get install google-perftools
!pip install reaver[tf-gpu,gym,atari,mujoco]

Download the headless SC2 client

In [0]:
!wget http://blzdistsc2-a.akamaihd.net/Linux/SC2.4.0.2.zip

Note: By typing in the password ‘iagreetotheeula’ you agree to be bound by the terms of [Blizzard's AI and Machine Learning License](https://blzdistsc2-a.akamaihd.net/AI_AND_MACHINE_LEARNING_LICENSE.html)

In [0]:
!unzip -P iagreetotheeula SC2.4.0.2.zip

In [0]:
!pip install pysc2

In [0]:

!git clone https://github.com/PaulConyngham/DeepRLBootCampLabs.git

In [0]:
!rsync -a ~/DeepRLBootCampLabs/pytorch ~/

In [0]:
!wget https://github.com/deepmind/pysc2/raw/master/pysc2/maps/mini_games/MoveToBeacon.SC2Map

In [0]:
!mkdir StarCraftII/Maps/mini_games


In [0]:
#!mkdir pytorch/data

!mkdir -p ./pytorch/data

In [0]:
!mv MoveToBeacon.SC2Map /content/StarCraftII/Maps/mini_games/

In [0]:
!ls /content/StarCraftII/Maps/mini_games/

In [0]:
!apt-get remove libtcmalloc*

## [end of installation section]

## Run the command below to get Pysc2 running on colabs!



In [0]:


!PYTHONPATH=. timeout -s 2 11h python ~/pytorch/sc2_agents/BaseTrainer.py --map=MoveToBeacon --train=True




## Now, Download the weights from your trained neural network!




In [0]:
# unfortunately, PySC2 uses Abseil, which treats python code as if its run like an app
# This does not play well with jupyter notebook
# So we will need to monkeypatch sys.argv


import sys
sys.argv = ["python", "--map", "Simple64"]

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS-IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Run an agent."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import importlib
import threading

from absl import app
from absl import flags
from future.builtins import range  # pylint: disable=redefined-builtin

from pysc2 import maps
from pysc2.env import available_actions_printer
from pysc2.env import run_loop
from pysc2.env import sc2_env
from pysc2.lib import point_flag
from pysc2.lib import stopwatch

FLAGS = flags.FLAGS

# because of Abseil's horrible design for running code underneath Colabs
# We have to pull out this ugly hack from the hat
if "flags_defined" not in globals():
    flags.DEFINE_bool("render", True, "Whether to render with pygame.")
    point_flag.DEFINE_point("feature_screen_size", "84",
                            "Resolution for screen feature layers.")
    point_flag.DEFINE_point("feature_minimap_size", "64",
                            "Resolution for minimap feature layers.")
    point_flag.DEFINE_point("rgb_screen_size", None,
                            "Resolution for rendered screen.")
    point_flag.DEFINE_point("rgb_minimap_size", None,
                            "Resolution for rendered minimap.")
    flags.DEFINE_enum("action_space", None, sc2_env.ActionSpace._member_names_,  # pylint: disable=protected-access
                      "Which action space to use. Needed if you take both feature "
                      "and rgb observations.")
    flags.DEFINE_bool("use_feature_units", False,
                      "Whether to include feature units.")
    flags.DEFINE_bool("disable_fog", False, "Whether to disable Fog of War.")

    flags.DEFINE_integer("max_agent_steps", 0, "Total agent steps.")
    flags.DEFINE_integer("game_steps_per_episode", None, "Game steps per episode.")
    flags.DEFINE_integer("max_episodes", 0, "Total episodes.")
    flags.DEFINE_integer("step_mul", 8, "Game steps per agent step.")

    flags.DEFINE_string("agent", "pysc2.agents.random_agent.RandomAgent",
                        "Which agent to run, as a python path to an Agent class.")
    flags.DEFINE_enum("agent_race", "random", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 1's race.")

    flags.DEFINE_string("agent2", "Bot", "Second agent, either Bot or agent class.")
    flags.DEFINE_enum("agent2_race", "random", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 2's race.")
    flags.DEFINE_enum("difficulty", "very_easy", sc2_env.Difficulty._member_names_,  # pylint: disable=protected-access
                      "If agent2 is a built-in Bot, it's strength.")

    flags.DEFINE_bool("profile", False, "Whether to turn on code profiling.")
    flags.DEFINE_bool("trace", False, "Whether to trace the code execution.")
    flags.DEFINE_integer("parallel", 1, "How many instances to run in parallel.")

    flags.DEFINE_bool("save_replay", True, "Whether to save a replay at the end.")

    flags.DEFINE_string("map", None, "Name of a map to use.")
    flags.mark_flag_as_required("map")

flags_defined = True

def run_thread(agent_classes, players, map_name, visualize):
  """Run one thread worth of the environment with agents."""
  with sc2_env.SC2Env(
      map_name=map_name,
      players=players,
      agent_interface_format=sc2_env.parse_agent_interface_format(
          feature_screen=FLAGS.feature_screen_size,
          feature_minimap=FLAGS.feature_minimap_size,
          rgb_screen=FLAGS.rgb_screen_size,
          rgb_minimap=FLAGS.rgb_minimap_size,
          action_space=FLAGS.action_space,
          use_feature_units=FLAGS.use_feature_units),
      step_mul=FLAGS.step_mul,
      game_steps_per_episode=FLAGS.game_steps_per_episode,
      disable_fog=FLAGS.disable_fog,
      visualize=visualize) as env:
    env = available_actions_printer.AvailableActionsPrinter(env)
    agents = [agent_cls() for agent_cls in agent_classes]
    run_loop.run_loop(agents, env, FLAGS.max_agent_steps, FLAGS.max_episodes)
    if FLAGS.save_replay:
      env.save_replay(agent_classes[0].__name__)


def main(unused_argv):
  """Run an agent."""
  stopwatch.sw.enabled = FLAGS.profile or FLAGS.trace
  stopwatch.sw.trace = FLAGS.trace

  map_inst = maps.get(FLAGS.map)

  agent_classes = []
  players = []

  agent_module, agent_name = FLAGS.agent.rsplit(".", 1)
  agent_cls = getattr(importlib.import_module(agent_module), agent_name)
  agent_classes.append(agent_cls)
  players.append(sc2_env.Agent(sc2_env.Race[FLAGS.agent_race]))

  if map_inst.players >= 2:
    if FLAGS.agent2 == "Bot":
      players.append(sc2_env.Bot(sc2_env.Race[FLAGS.agent2_race],
                                 sc2_env.Difficulty[FLAGS.difficulty]))
    else:
      agent_module, agent_name = FLAGS.agent2.rsplit(".", 1)
      agent_cls = getattr(importlib.import_module(agent_module), agent_name)
      agent_classes.append(agent_cls)
      players.append(sc2_env.Agent(sc2_env.Race[FLAGS.agent2_race]))

  threads = []
  for _ in range(FLAGS.parallel - 1):
    t = threading.Thread(target=run_thread,
                         args=(agent_classes, players, FLAGS.map, False))
    threads.append(t)
    t.start()

  run_thread(agent_classes, players, FLAGS.map, FLAGS.render)

  for t in threads:
    t.join()

  if FLAGS.profile:
    print(stopwatch.sw)


def entry_point():  # Needed so setup.py scripts work.
  app.run(main)


if __name__ == "__main__":
  app.run(main)

In [0]:
from google.colab import files

files.download('./pytorch/data/SC2QAgent')

In [0]:
# and thats it!